In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
batch_size=32
classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [3]:
(train_images,train_labels),(test_images,test_labels)=tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 6s 0us/step


In [4]:
def img_process(img):
  img=img.astype('float')
  img=tf.keras.applications.resnet50.preprocess_input(img)
  return img

In [5]:
x_train=img_process(train_images)
x_test=img_process(test_images)

In [6]:
print("Shape of X_train : ",x_train.shape)
print("Shape of training_labels : ",train_labels.shape)
print("Shape of X_val : ",x_test.shape)
print("Shape of validation_labels : ",test_labels.shape)

Shape of X_train :  (50000, 32, 32, 3)
Shape of training_labels :  (50000, 1)
Shape of X_val :  (10000, 32, 32, 3)
Shape of validation_labels :  (10000, 1)


In [7]:
def predefine_model(inputs):
  resnet=tf.keras.applications.resnet.ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))
  x=resnet(inputs)
  return x

In [8]:
def output_layer(inputs):
  x=predefine_model(inputs)
  x=tf.keras.layers.GlobalAveragePooling2D()(x)
  x=tf.keras.layers.Flatten()(x)
  x=tf.keras.layers.Dense(1024,activation='relu')(x)
  x=tf.keras.layers.Dense(512,activation='relu')(x)
  x=tf.keras.layers.Dense(10,activation='softmax',name='classification')(x)
  return x

In [15]:
def get_model():
  input=tf.keras.layers.Input(shape=(32,32,3))
  x=tf.keras.layers.UpSampling2D(size=(7,7))(input)
  x=output_layer(x)
  model=tf.keras.models.Model(inputs=input,outputs=x)
  model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01),loss=tf.keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])
  return model

In [16]:
model=get_model()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [17]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_5 (Dense)              (None, 512)               5248

In [18]:
history=model.fit(x_train,train_labels,epochs=4,batch_size=32,verbose=1,validation_data=(x_test,test_labels),validation_batch_size=32)

Epoch 1/4
1563/1563 [==============================] - 1158s 719ms/step - loss: 0.3482 - accuracy: 0.8832 - val_loss: 0.1804 - val_accuracy: 0.9391
Epoch 2/4
1563/1563 [==============================] - 1121s 717ms/step - loss: 0.0965 - accuracy: 0.9675 - val_loss: 0.1538 - val_accuracy: 0.9500
Epoch 3/4
1563/1563 [==============================] - 1124s 719ms/step - loss: 0.0389 - accuracy: 0.9868 - val_loss: 0.1804 - val_accuracy: 0.9455
Epoch 4/4
1563/1563 [==============================] - 1121s 717ms/step - loss: 0.0207 - accuracy: 0.9936 - val_loss: 0.1521 - val_accuracy: 0.9565
